##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Search re-ranking using Gemini embeddings

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Search_reranking_using_embeddings.ipynb"><img src="../images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

This notebook demonstrates the use of embeddings to re-rank search results. This walkthrough will focus on the following objectives:



1.   Setting up your development environment and API access to use Gemini.
2.   Using Gemini's function calling support to access the Wikipedia API.
3.   Embedding content via Gemini API.
4.   Re-ranking the search results.


This is how you will implement search re-ranking:


1.   The user will make a search query.
2.   You will use Wikipedia API to return the relevant search results.
3.   The search results will be embedded and their relevance will be evaluated by calculating distance metrics like cosine similarity.
4.   The most relevant search result will be returned as the final answer.

> The non-source code materials in this notebook are licensed under Creative Commons - Attribution-ShareAlike CC-BY-SA 4.0, https://creativecommons.org/licenses/by-sa/4.0/legalcode.

## Setup


In [ ]:
!pip install -q -U "google-generativeai>=0.7.2"

In [ ]:
!pip install -q wikipedia

Note: The [`wikipedia` package](https://pypi.org/project/wikipedia/) notes that it was "designed for ease of use and simplicity, not for advanced use", and that production or heavy use should instead "use [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) or one of the other more advanced [Python MediaWiki API wrappers](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python)".

In [ ]:
import json
import textwrap

import google.generativeai as genai

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

import numpy as np

from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Define tools

As stated earlier, this tutorial uses Gemini's function calling support to access the Wikipedia API. Please refer to the [docs](https://ai.google.dev/docs/function_calling) to learn more about function calling.

### Define the search function

To cater to the search engine needs, you will design this function in the following way:


*   For each search query, the search engine will use the `wikipedia.search` method to get relevant topics.
*   From the relevant topics, the engine will choose `n_topics(int)` top candidates and will use `gemini-1.5-flash` to extract relevant information from the page.
*   The engine will avoid duplicate entries by maintaining a search history.


In [2]:
def wikipedia_search(search_queries: list[str]) -> list[str]:
  """Search wikipedia for each query and summarize relevant docs."""
  n_topics=3
  search_history = set() # tracking search history
  search_urls = []
  mining_model = genai.GenerativeModel('gemini-1.5-flash')
  summary_results = []

  for query in search_queries:
    print(f'Searching for "{query}"')
    search_terms = wikipedia.search(query)

    print(f"Related search terms: {search_terms[:n_topics]}")
    for search_term in search_terms[:n_topics]: # select first `n_topics` candidates
      if search_term in search_history: # check if the topic is already covered
        continue

      print(f'Fetching page: "{search_term}"')
      search_history.add(search_term) # add to search history

      try:
        # extract the relevant data by using `gemini-1.5-flash` model
        page = wikipedia.page(search_term, auto_suggest=False)
        url = page.url
        print(f"Information Source: {url}")
        search_urls.append(url)
        page = page.content
        response = mining_model.generate_content(textwrap.dedent(f"""\
            Extract relevant information
            about user's query: {query}
            From this source:

            {page}

            Note: Do not summarize. Only Extract and return the relevant information
        """))

        urls = [url]
        if response.candidates[0].citation_metadata:
          extra_citations = response.candidates[0].citation_metadata.citation_sources
          extra_urls = [source.url for source in extra_citations]
          urls.extend(extra_urls)
          search_urls.extend(extra_urls)
          print("Additional citations:", response.candidates[0].citation_metadata.citation_sources)
        try:
          text = response.text
        except ValueError:
          pass
        else:
          summary_results.append(text + "\n\nBased on:\n  " + ',\n  '.join(urls))

      except DisambiguationError:
        print(f"""Results when searching for "{search_term}" (originally for "{query}")
        were ambiguous, hence skipping""")
        continue

      except PageError:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue
        
      except:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

  print(f"Information Sources:")
  for url in search_urls:
    print('    ', url)

  return summary_results


In [3]:
example = wikipedia_search(["What are LLMs?"])

Searching for "What are LLMs?"
Related search terms: ['Large language model', 'Stochastic parrot', 'Prompt engineering']
Fetching page: "Large language model"
Information Source: https://en.wikipedia.org/wiki/Large_language_model
Large language model did not match with any page id, hence skipping.
Fetching page: "Stochastic parrot"
Information Source: https://en.wikipedia.org/wiki/Stochastic_parrot
Fetching page: "Prompt engineering"
Information Source: https://en.wikipedia.org/wiki/Prompt_engineering
Prompt engineering did not match with any page id, hence skipping.
Information Sources:
     https://en.wikipedia.org/wiki/Large_language_model
     https://en.wikipedia.org/wiki/Stochastic_parrot
     https://en.wikipedia.org/wiki/Prompt_engineering


Here is what the search results look like:

In [4]:
from IPython.display import display

for e in example:
  display(to_markdown(e))

> ## Relevant information about LLMs from the source:
> 
> * **LLMs are described as "stochastic parrots"**: This means they can generate plausible language but lack understanding of the meaning of the words they process.
> * **The term was coined in 2021**:  By Emily Bender, Timnit Gebru, Angelina McMillan-Major, and Margaret Mitchell in their paper "On the Dangers of Stochastic Parrots: Can Language Models Be Too Big?" 🦜.
> * **LLMs have limitations**: They are limited by the data they are trained on and may produce dangerous outputs due to biases or limitations in the training data.
> * **LLMs don't understand the meaning of language**: They only statistically link words and sentences together, unable to differentiate facts and fiction.
> * **The debate about LLMs understanding is ongoing**: While some researchers argue LLMs are merely pattern matchers, others believe they are capable of understanding language.
> * **LLMs' performance on benchmarks and experiments is contested**: Some argue their performance proves understanding, while others say they are exploiting shortcuts and correlations within the data.
> * **The "stochastic parrot" concept is widely used**: It's a popular term among AI skeptics and has been adopted by some AI professionals to describe LLMs' limitations. 
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/Stochastic_parrot

### Pass the tools to the model

If you pass a list of functions to the `GenerativeModel`'s `tools` argument,
it will extract a schema from the function's signature and type hints, and then pass schema along to the API calls. In response the model may return a `FunctionCall` object asking to call the function.

Note: This approach only handles annotations of `AllowedTypes = int | float | str | dict | list['AllowedTypes']`

The `GenerativeModel` will keep a reference to the function inself, so that it _can_ execute the function locally later.

In [5]:
model = genai.GenerativeModel(
    'gemini-1.5-flash',
    tools=[wikipedia_search],
    generation_config={'temperature': 0.6})

## Generate supporting search queries

In order to have multiple supporting search queries to the user's original query, you will ask the model to generate more such queries. This would help the engine to cover the asked question on comprehensive levels.

In [6]:
instructions = """You have access to the Wikipedia API which you will be using
to answer a user's query. Your job is to generate a list of search queries which
might answer a user's question. Be creative by using various key-phrases from
the user's query. To generate variety of queries, ask questions which are
related to  the user's query that might help to find the answer. The more
queries you generate the better are the odds of you finding the correct answer.
Here is an example:

user: Tell me about Cricket World cup 2023 winners.

function_call: wikipedia_search(['What is the name of the team that
won the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup
2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What
was the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',
'Who lifted the Cricket World Cup 2023 trophy?'])

The search function will return a list of article summaries, use these to
answer the  user's question.

Here is the user's query: {query}
"""

In order to yield creative and a more random variety of questions, you will set the model's temperature parameter to a value higher. Values can range from [0.0,1.0], inclusive. A value closer to 1.0 will produce responses that are more varied and creative, while a value closer to 0.0 will typically result in more straightforward responses from the model.

## Enable automatic function calling and call the API

Now start a new chat with `enable_automatic_function_calling=True`. With it enabled, the `genai.ChatSession` will handle the back and forth required to call the function, and return the final response:

In [7]:
model = genai.GenerativeModel(
    'gemini-1.5-flash', tools=[wikipedia_search], generation_config={'temperature': 0.6})

chat = model.start_chat(enable_automatic_function_calling=True)

query = "Explain how deep-sea life survives."

res = chat.send_message(instructions.format(query=query))

Searching for "How do deep-sea creatures survive?"
Related search terms: ['Deep-sea community', 'Deep sea', 'Deep-sea fish']
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Searching for "What are the adaptations of deep-sea animals?"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Deep-sea community']
Searching for "Deep-sea life survival strategies"
Related search terms: ['Anglerfish', 'Thalassophobia', 'Brine pool']
Fetching page: "Anglerfish"
Information Source: https://en.wikipedia.org/wiki/Anglerfish
Fetching page: "Thalassophobia"
Information Source: https://en.wikipedia.org/wiki/Thalassophobia
Fetching page: "Brine pool"
Information Source: https://en.wikipedia.org/wiki/Brine_pool
Searching for "How do deep-sea animals get food?"
Related se

In [8]:
to_markdown(res.text)

> Deep-sea life faces unique challenges due to the extreme environment. Here's how they survive:
> 
> * **Food Scarcity:**  The deep sea has limited food sources. Creatures rely on marine snow (sinking organic matter), whale falls (dead whale carcasses), and chemosynthesis (using chemicals from hydrothermal vents or cold seeps for energy).
> * **Pressure:** Deep-sea animals have evolved to withstand immense pressure. They have small sizes, gelatinous flesh, minimal skeletal structure, and lack internal cavities that could collapse.
> * **Darkness:**  The lack of sunlight has led to adaptations like large eyes for visual hunting and communication, bioluminescence for camouflage, and vertical migrations for food in the twilight zone. In the midnight zone, organisms have reduced or absent organs, relying on detritus for food. 
> * **Temperature:**  While generally cold and stable, deep-sea temperatures vary near hydrothermal vents, where extreme changes occur.
> * **Salinity:**  Deep-sea salinity is generally constant, except in areas like the Mediterranean Sea and brine pools.
> 
> Deep-sea creatures have evolved remarkable adaptations to thrive in this challenging environment. 


Check for additional citations:

In [9]:
res.candidates[0].citation_metadata or 'No citations found'

'No citations found'

That looks like it worked. You can go through the chat history to see the details of what was sent and received in the function calls:

In [10]:
for content in chat.history:
  part = content.parts[0]

  print(f'{content.role} -> ', end='')
  print(json.dumps(type(part).to_dict(part), indent=2))
  print('---' * 20)


user -> {
  "text": "You have access to the Wikipedia API which you will be using\nto answer a user's query. Your job is to generate a list of search queries which\nmight answer a user's question. Be creative by using various key-phrases from\nthe user's query. To generate variety of queries, ask questions which are\nrelated to  the user's query that might help to find the answer. The more\nqueries you generate the better are the odds of you finding the correct answer.\nHere is an example:\n\nuser: Tell me about Cricket World cup 2023 winners.\n\nfunction_call: wikipedia_search(['What is the name of the team that\nwon the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup\n2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What\nwas the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',\n'Who lifted the Cricket World Cup 2023 trophy?'])\n\nThe search function will return a list of article summaries, use these to\nans

In the chat history you can see all 4 steps:

1. The user sent the query.
2. The model replied with a `genai.protos.FunctionCall` calling the `wikipedia_search` with a number of relevant searches.
3. Because you set `enable_automatic_function_calling=True` when creating the `genai.ChatSession`, it  executed the search function and returned the list of article summaries to the model.
4. Folliwing the instructions in the prompt, the model generated a final answer based on those summaries.


## [Optional] Manually execute the function call

If you want to understand what happened behind the scenes, this section executes the `FunctionCall` manually to demonstrate.

In [12]:
chat = model.start_chat()

In [13]:
result = chat.send_message(instructions.format(query=query))

Initially the model returns a FunctionCall:

In [14]:
fc = result.candidates[0].content.parts[0].function_call
fc = type(fc).to_dict(fc)
print(json.dumps(fc, indent=2))

{
  "name": "wikipedia_search",
  "args": {
    "search_queries": [
      "How do deep-sea creatures survive?",
      "What are the adaptations of deep-sea life?",
      "Deep-sea life survival mechanisms",
      "Challenges of deep-sea survival",
      "Deep-sea environment and its effect on life",
      "How do deep-sea animals find food?",
      "How do deep-sea animals reproduce?",
      "Deep-sea food web",
      "Adaptations of deep-sea fish",
      "Deep-sea ecosystems"
    ]
  }
}


In [15]:
fc['name']

'wikipedia_search'

Call the function with generated arguments to get the results.

In [16]:
summaries = wikipedia_search(**fc['args'])

Searching for "How do deep-sea creatures survive?"
Related search terms: ['Deep-sea community', 'Deep sea', 'Deep-sea fish']
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Searching for "What are the adaptations of deep-sea life?"
Related search terms: ['Deep sea', 'Deep-sea fish', 'The Deep Blue Sea (play)']
Fetching page: "The Deep Blue Sea (play)"
Information Source: https://en.wikipedia.org/wiki/The_Deep_Blue_Sea_(play)
Searching for "Deep-sea life survival mechanisms"
Related search terms: ['Deep-sea community', 'Hydrothermal vent', 'Deep sea mining']
Fetching page: "Hydrothermal vent"
Information Source: https://en.wikipedia.org/wiki/Hydrothermal_vent
Fetching page: "Deep sea mining"
Information Source: https://en.wikipedia.org/wiki/Deep_sea_mini

Now send the `FunctionResult` to the model.

In [17]:
response = chat.send_message(
    genai.protos.Content(
      parts=[genai.protos.Part(
          function_response = genai.protos.FunctionResponse(
            name='wikipedia_search',
            response={'result': summaries}
          )
      )]
    )
)

to_markdown(response.text)

> Deep-sea life faces unique challenges to survive, including immense pressure, darkness, and limited food.  Here's what I found:
> 
> * **Adaptations:** Deep-sea creatures have evolved a variety of adaptations to survive in these extreme conditions. These include bioluminescence for communication and attracting prey, specialized senses to navigate in the dark, and slow metabolisms to conserve energy.
> * **Food Sources:**  Food is scarce in the deep sea, so many creatures are scavengers, feeding on dead organisms that fall from above.  Some rely on chemosynthesis, where bacteria use chemicals from hydrothermal vents to produce energy.
> * **Challenges of Exploration:** Deep-sea exploration is challenging due to the extreme conditions and the cost of research.  This means we still have much to learn about how life survives in these depths. 


## Re-ranking the search results

Helper function to embed the content:

In [18]:
def get_embeddings(content: list[str]) -> np.ndarray:
  embeddings = genai.embed_content('models/embedding-001', content, 'SEMANTIC_SIMILARITY')
  embds = embeddings.get('embedding', None)
  embds = np.array(embds).reshape(len(embds), -1)
  return embds

Please refer to the [embeddings guide](https://ai.google.dev/docs/embeddings_guide) for more information on embeddings.

Your next step is to define functions that you can use to calculate similarity scores between two embedding vectors. These scores will help you decide which embedding vector is the most relevant vector to the user's query.


You will now implement cosine similarity as your metric. Here returned embedding vectors will be of unit length and hence their L1 norm (`np.linalg.norm()`) will be ~1. Hence, calculating cosine similarity is esentially same as calculating their dot product score.

In [19]:
def dot_product(a: np.ndarray, b: np.ndarray):
  return (a @ b.T)

### Similarity with user's query

Now it's time to find the most relevant search result returned by the Wikipedia API.

Use Gemini API to get embeddings for user's query and search results.

In [20]:
search_res = get_embeddings(summaries)
embedded_query = get_embeddings([query])

Calculate similarity score:

In [21]:
sim_value = dot_product(search_res, embedded_query)

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [22]:
print(summaries[np.argmax(sim_value)])

## Deep-sea Life Survival Mechanisms:

This document focuses on deep-sea mining and its environmental impacts, not on deep-sea life survival mechanisms. It does not provide information about the adaptations and strategies used by organisms to survive in the harsh conditions of the deep sea. 


Based on:
  https://en.wikipedia.org/wiki/Deep_sea_mining


### Similarity with Hypothetical Document Embeddings (HyDE)

Drawing inspiration from [Gao et al](https://arxiv.org/abs/2212.10496) the objective here is to generate a template answer to the user's query using `gemini-1.5-flash`'s internal knowledge. This hypothetical answer will serve as a baseline to calculate relevance of all the search results.

In [23]:
hypothetical_ans_model = genai.GenerativeModel('gemini-1.5-flash')
res = hypothetical_ans_model.generate_content(f"""Generate a hypothetical answer
to the user's query by using your own knowledge. Assume that you know everything
about the said topic. Do not use factual information, instead use placeholders
to complete your answer. Your answer should feel like it has been written by a human.

query: {query}""")

to_markdown(res.text)

> Life in the deep sea is a fascinating world of adaptation and resilience. These creatures have had to develop incredible ways to survive in a place where sunlight doesn't reach, pressure is immense, and food is scarce.
> 
> First, there's the issue of **[food source]**.  Since photosynthesis is impossible at those depths, deep-sea life relies on **[alternative food source]**. Some creatures are **[describe feeding strategy]**, while others are **[describe another feeding strategy]**. This makes the food web incredibly intricate and delicate.
> 
> Then there's the **[pressure]**. The deep sea is like a giant weight pressing down on everything. To survive, creatures have developed **[adaptation related to pressure]**, which helps them thrive in such extreme conditions. 
> 
> And let's not forget the **[temperature]**.  It's **[describe temperature]**, but some creatures can tolerate even **[describe extreme temperature]**.  They have **[adaptation related to temperature]** to make sure they can survive in these frigid waters.
> 
> Finally, the **[lack of light]**.  Without sunlight, it's a world of **[describe darkness]**.  Many creatures have **[adaptation related to darkness]**, like **[describe specific adaptation]**, to help them navigate and find their way around.
> 
> All these adaptations, from their unique feeding habits to their incredible resistance to pressure, make deep-sea life one of the most fascinating and resilient ecosystems on Earth. It's a world of constant challenges, but also of incredible beauty and wonder. 


Use Gemini API to get embeddings for the baseline answer and compare them with search results

In [24]:
hypothetical_ans = get_embeddings([res.text])

Calculate similarity scores to rank the search results

In [25]:
sim_value = dot_product(search_res, hypothetical_ans)

In [26]:
sim_value

array([[0.83514002],
       [0.83081487],
       [0.84890951],
       [0.70791981],
       [0.78692995],
       [0.79417004],
       [0.78864686],
       [0.74766133],
       [0.7662298 ],
       [0.87420404],
       [0.72367365],
       [0.75990845],
       [0.75220328]])

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [27]:
to_markdown(summaries[np.argmax(sim_value)])

> Deep-sea animals find food in various ways depending on their habitat and the type of food available. Some deep-sea animals are scavengers, feeding on dead animals and plants that fall to the ocean floor. Others are predators, hunting and capturing live prey. Some deep-sea animals have special adaptations that allow them to survive in the dark, cold, and high-pressure environment of the deep ocean. For example, some have bioluminescent organs that allow them to attract prey or mates. Others have long, sensitive tentacles that allow them to sense prey in the dark.
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/Marine_life

You have now created a search re-ranking engine using embeddings!

## Next steps

I hope you found this example helpful! Check out more examples in the [Gemini Guide](https://github.com/google-gemini/gemini-guide/) to learn more.